## Testset Generation for RAG

[Open in google colab](https://colab.research.google.com/github/explodinggradients/ragas/blob/main/docs/getstarted/rag_testset_generation.ipynb)


### Load Sample Documents

For the sake of this tutorial we will use sample documents from this [repository](https://huggingface.co/datasets/explodinggradients/Sample_Docs_Markdown). You can replace this with your own documents.


In [13]:
! git clone https://huggingface.co/datasets/explodinggradients/Sample_Docs_Markdown

Cloning into 'Sample_Docs_Markdown'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 10 (delta 0), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (10/10), 103.01 KiB | 4.68 MiB/s, done.


### Load documents
Now we will load the documents from the sample dataset using `DirectoryLoader`, which is one of document loaders from [langchain_community](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/). You may also use any loaders from [llama_index](https://docs.llamaindex.ai/en/stable/understanding/loading/llamahub/)

In [7]:
from langchain_community.document_loaders import DirectoryLoader

In [15]:
path = "Sample_Docs_Markdown/"
loader = DirectoryLoader(path, glob="**/*.md")
docs = loader.load()


### Choose your LLM

You may choose to use any [LLM of your choice]()

=== "OpenAI"
    This guide utilizes OpenAI for running some metrics, so ensure you have your OpenAI key ready and available in your environment.
    
    ```python
    import os
    os.environ["OPENAI_API_KEY"] = "your-openai-key"
    ```
    Wrapp the LLMs in `LangchainLLMWrapper`
    ```python
    from ragas.llms import LangchainLLMWrapper
    from langchain_openai import ChatOpenAI
    generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
    ```




=== "AWS Bedrock"
    First you have to set your AWS credentials and configurations
    
    ```python
    config = {
        "credentials_profile_name": "your-profile-name",  # E.g "default"
        "region_name": "your-region-name",  # E.g. "us-east-1"
        "model_id": "your-model-id",  # E.g "anthropic.claude-v2"
        "model_kwargs": {"temperature": 0.4},
    }
    ```
    define you LLMs
    ```python
    from langchain_aws.chat_models import BedrockChat
    from ragas.llms import LangchainLLMWrapper
    generator_llm = LangchainLLMWrapper(BedrockChat(
        credentials_profile_name=config["credentials_profile_name"],
        region_name=config["region_name"],
        endpoint_url=f"https://bedrock-runtime.{config['region_name']}.amazonaws.com",
        model_id=config["model_id"],
        model_kwargs=config["model_kwargs"],
    ))
    ```

### Run test generation

Now we will run the test generation using the loaded documents and the LLM setup. If you have used `llama_index` to load documents, please use `generate_with_llama_index_docs` method instead.

In [4]:
from ragas.experimental.testset import TestsetGenerator

In [16]:
generator = TestsetGenerator(llm=generator_llm)
dataset = generator.generate_with_langchain_docs(docs, test_size=10)

### Export

You may now export and inspect the generated testset.

In [12]:
dataset.to_hf_dataset().to_pandas()

,user_input,reference_contexts,reference
0,What strategies does DeepSeekMoE employ to ach...,[1. Introduction\n\nRecent research and practi...,DeepSeekMoE employs two principal strategies t...
1,How Mixture-Of-Experts architecture make Trans...,[2. Preliminaries: Mixture-Of-Experts For Tran...,The Mixture-Of-Experts (MoE) architecture make...
2,How the DeepSeekMoE architecture optimize expe...,[3. Deepseekmoe Architecture\n\nOn top of the ...,The DeepSeekMoE architecture optimizes expert ...
3,What are the key components and strategies inv...,[4. Validation Experiments 4.1. Experimental S...,The key components and strategies involved in ...
4,How DeepSeekMoE 16B perform better with effici...,[5. Scaling Up To Deepseekmoe 16B\n\nWith the ...,DeepSeekMoE 16B performs better with efficient...
5,How does DeepSeekMoE 16B achive resorce-effici...,[6. Alignment For Deepseekmoe 16B\n\nPrevious ...,DeepSeekMoE 16B achieves resource-efficient pe...
6,What factors contribute to performance and eff...,[7. Deepseekmoe 145B Ongoing\n\nEncouraged by ...,The performance and efficiency of DeepSeekMoE ...
7,What are the key advancements in Mixture of Ex...,[8. Related Work\n\nThe Mixture of Experts (Mo...,Key advancements in Mixture of Experts (MoE) t...
8,What key features and advantages of DeepSeekMo...,"[9. Conclusion\n\nIn this paper, we introduce ...",Key features and advantages of DeepSeekMoE in ...
9,How does DeepSeekMoE demonstrate competitive p...,[Appendices A. Overview Of Hyper-Parameters\n\...,DeepSeekMoE demonstrates competitive performan...
